<a href="https://colab.research.google.com/github/tecXworld/Natural-Language-Processing/blob/main/Sentiment_Analysis_Imdb_reviews_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importing tensorflow libraries
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [2]:
#checking the tensorflow version
tf.__version__

'2.5.0'

In [ ]:
#updrade tensorflow
!pip install tensorflow --upgrade

In [4]:
#list of datasets included in tensorflow
tfds.list_builders()

['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'arc',
 'bair_robot_pushing_small',
 'bccd',
 'beans',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'blimp',
 'bool_q',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clic',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coco_captions',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'coqa',
 'cos_e',
 'cosmos_qa',
 'covid19sum',
 'crema_d',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'deep_weeds',
 'definite_pronoun_resolution',
 'dementiabank',
 'diabetic_retinopathy_detection',
 'div2k',
 'dmlab',
 'downsampled_imagenet',
 

In [5]:
train_data , test_data = tfds.load(name='imdb_reviews' , split = (tfds.Split.TRAIN,tfds.Split.TEST),as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteEV42XU/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteEV42XU/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteEV42XU/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [6]:
#Checking the type of a dataset
type(train_data)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [7]:
#coverting a dataset into list

new_train_data = list(train_data)
print(new_train_data[0])

(<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


In [8]:
#extract the information from the dataset in the form of batches
train_example_data , train_label_data = next(iter(train_data.batch(10)))

In [9]:
print(train_example_data)

tf.Tensor(
[b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
 b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot de

In [10]:
print(train_label_data)

tf.Tensor([0 0 0 1 1 1 0 0 0 0], shape=(10,), dtype=int64)


In [11]:
#for the converrsion of word to vec
#uses tensor flow hub for conversion of words to vectors
pre_trained_model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1"
hub_layer = hub.KerasLayer(pre_trained_model , input_shape=[] , dtype=tf.string,trainable=True)

In [12]:

hub_layer(train_example_data[:2])

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[ 1.668895  , -3.918618  ,  3.7993348 , -1.4097565 , -3.3214598 ,
        -0.9996616 , -2.0233727 ,  1.2953283 ,  5.0233593 , -1.0990871 ,
        -2.0879953 , -1.2672297 , -0.18820252, -0.07738297, -2.9172704 ,
         1.7943126 ,  1.2375495 , -1.5559573 , -2.4811747 , -0.7813505 ],
       [ 1.7287169 , -2.4736629 ,  3.3280423 ,  0.9174705 , -4.062213  ,
        -4.791508  , -2.7125058 ,  1.2155541 ,  3.7063696 , -0.95357007,
        -1.855471  ,  1.2775491 , -0.07673654,  0.6835056 , -4.822123  ,
         2.6815715 ,  4.479097  , -2.360208  , -3.2802453 , -1.5463475 ]],
      dtype=float32)>

In [13]:
#creating a sequential model with two dense layer and one hub_layer
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16,activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                389380    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 389,733
Trainable params: 389,733
Non-trainable params: 0
_________________________________________________________________


In [14]:
#compilation of model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [15]:
#fitting the data of dataset into  the model the form of batch
model.fit(train_data.shuffle(10000).batch(512),epochs=20,verbose=1)

Epoch 1/20
49/49 [==============================] - 4s 59ms/step - loss: 0.8898 - accuracy: 0.5404
Epoch 2/20
49/49 [==============================] - 3s 58ms/step - loss: 0.6441 - accuracy: 0.6352
Epoch 3/20
49/49 [==============================] - 3s 58ms/step - loss: 0.6182 - accuracy: 0.6646
Epoch 4/20
49/49 [==============================] - 3s 58ms/step - loss: 0.5899 - accuracy: 0.6935
Epoch 5/20
49/49 [==============================] - 3s 58ms/step - loss: 0.5558 - accuracy: 0.7275
Epoch 6/20
49/49 [==============================] - 3s 56ms/step - loss: 0.5148 - accuracy: 0.7596
Epoch 7/20
49/49 [==============================] - 3s 57ms/step - loss: 0.4679 - accuracy: 0.7918
Epoch 8/20
49/49 [==============================] - 3s 57ms/step - loss: 0.4172 - accuracy: 0.8224
Epoch 9/20
49/49 [==============================] - 3s 59ms/step - loss: 0.3640 - accuracy: 0.8485
Epoch 10/20
49/49 [==============================] - 3s 58ms/step - loss: 0.3205 - accuracy: 0.8696
Epoch 11/

In [16]:
#prediction
model.predict(["this is the great movie i have ever seen"])

array([[0.961096]], dtype=float32)